In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
inputs = layers.Input((None,224,224,3))
mobilenet = tf.keras.applications.mobilenet.MobileNet(input_shape=(224,224,3), alpha=0.5, include_top=False, weights='imagenet')
net = layers.TimeDistributed(mobilenet, name="mn")(inputs)
net = layers.ConvLSTM2D(50,3, return_sequences=True)(net)
net = layers.TimeDistributed(layers.Flatten())(net)
net = layers.TimeDistributed(layers.Dense(10, activation="softmax"))(net)
model = Model(inputs,net)

for layer in mobilenet.layers:
    layer.trainable = False

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss="binary_crossentropy",
              metrics=['accuracy']
)
model.summary()

5578752/5577668 [==============================] - 6s 1us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, 224, 224, 3) 0         
_________________________________________________________________
mn (TimeDistributed)         (None, None, 5, 5, 512)   829536    
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 3, 3, 50)    1011800   
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 450)         0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 1)           451       
Total params: 1,841,787
Trainable params: 1,012,251
Non-trainable params: 829,536
_________________________________________________________________


In [36]:
def load_example(example):
    # need to modify this function to have variable number of timesteps
    example["image"] = tf.image.resize(example["image"], (224,224))
    example["image"] = tf.stack([example["image"], example["image"], example["image"]])
    
    example["label"] = tf.stack([example["label"],example["label"],example["label"]])
    example["label"] = tf.expand_dims(example["label"], axis=-1)
    
    data = example["image"]
    label = example["label"]

    return data, label
    

In [37]:
def create_datasets():
    ds_train = tfds.load(name="cats_vs_dogs", split=tfds.Split.TRAIN.subsplit(tfds.percent[:90]))
    ds_train = ds_train.map(load_example).repeat().batch(16)
    #ds_train = ds_train.batch(16)
    #ds_train = ds_train.repeat()

    ds_test = tfds.load(name="cats_vs_dogs", split=tfds.Split.TRAIN.subsplit(tfds.percent[91:]))
    ds_test = ds_test.map(load_example)
    ds_test = ds_test.batch(16)
    ds_test = ds_test.repeat()
    
    
    return ds_train, ds_test

In [38]:
ds_train, ds_test = create_datasets()
#for example in ds_train.take(1):
#    print(example[1])

In [39]:
model.fit(ds_train, epochs=3, steps_per_epoch=1309 ,validation_data=ds_test, validation_steps=131)

Epoch 1/3
 571/1309 [============>.................] - ETA: 45:06 - loss: 8.0340 - acc: 0.4961

KeyboardInterrupt: 